In [58]:
import json
import gensim
from gensim.models.fasttext import FastText
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
import pandas as pd
import re
import numpy as np

In [2]:
with open ("Lyrics_analysis-master\Lyrics_analysis-master\songs.json", 'r') as f:
    songs = json.load(f)

In [77]:
'''def max_len(lyrics):
    max_len = 0
    for i in range(len(lyrics)):
        print(lyrics)
        if len(lyrics[i]['lyrics']) > max_len:
            max_len = len(lyrics[i]['lyrics'])
            
    return max_len'''

print(len(songs))
print(songs[0])
limit = len(songs)

bad = []

max_len = 0
song = ""

for i in range(limit):
    try:
        if len(songs[i]['lyrics']) > max_len:
            max_len = len(songs[i]['lyrics'])
            song = songs[i]
    except TypeError:
        bad.append(i)
        max_len = max_len
    except KeyError:
        bad.append(i)
        max_len = max_len

8621
{'url': 'https://www.lyrics.com/lyric/35715164/Lewis+Capaldi/Someone+You+Loved', 'lyrics': ["I'm going under and this time I fear there's no one to save me", 'This all or nothing really got a way of driving me crazy', 'I need somebody to heal', 'Somebody to know', 'Somebody to have', 'Somebody to hold', "It's easy to say", "But it's never the same", 'I guess I kinda liked the way you numbed all the pain', '', 'Now the day bleeds', 'Into nightfall', "And you're not here", 'To get me through it all', 'I let my guard down', 'And then you pulled the rug', 'I was getting kinda used to being someone you loved', '', "I'm going under and this time I fear there's no one to turn to", 'This all or nothing way of loving got me sleeping without you', 'Now, I need somebody to know', 'Somebody to heal', 'Somebody to have', 'Just to know how it feels', "It's easy to say but it's never the same", 'I guess I kinda liked the way you helped me escape', '', 'Now the day bleeds', 'Into nightfall', "And

In [79]:
titles = []
lyrics = []
bad2 = []

for i in range(limit):
    try:
        titles.append(songs[i]['title'])
        lyrics.append(songs[i]['lyrics'])
    except TypeError:
        bad2.append(i)
    except KeyError:
        bad2.append(i)
        max_len = max_len
        
assert(bad2 == bad)

df = pd.DataFrame({'songs' : titles, 'lyrics' : lyrics})
df.head()

,songs,lyrics
0,Someone You Loved,[I'm going under and this time I fear there's ...
1,Circles,"[Oh, oh, oh, Oh, oh, oh, Oh, oh, oh, Oh, oh, ,..."
2,Good as Hell [Two Stacks Remix],"[I do my hair toss, Check my nails, Baby how y..."
3,Truth Hurts,"[Why men great 'til they gotta be great?, Woo,..."
4,Lose You to Love Me,"[You promised the world and I fell for it, I p..."


In [80]:
stemmer = WordNetLemmatizer()
en_stop = set(nltk.corpus.stopwords.words('english'))

def preprocess(line):  
    
    line = re.sub(r'\W', ' ', str(line))
    line = re.sub(r'\s+[a-zA-Z]\s+', ' ', line)
    line = re.sub(r'\^[a-zA-Z]\s+', ' ', line)
    line = re.sub(r'\s+', ' ', line, flags=re.I)
    
    line = line.lower()
    
    tokens = line.split()    
    tokens = [stemmer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if word not in en_stop]
    #tokens = [word for word in tokens if len(word) > 3]
    
    return tokens
    
def preprocess_all(lyrics):
    ret = []
    for line in lyrics:
        tokens = preprocess(line)
        if not tokens == []:
            ret.append(tokens)
            
    return ret

In [81]:
df['preprocessed_lyrics'] = df['lyrics'].apply(preprocess_all)
df.head()

,songs,lyrics,preprocessed_lyrics
0,Someone You Loved,[I'm going under and this time I fear there's ...,"[[going, time, fear, one, save], [nothing, rea..."
1,Circles,"[Oh, oh, oh, Oh, oh, oh, Oh, oh, oh, Oh, oh, ,...","[[oh, oh, oh], [oh, oh, oh], [oh, oh, oh], [oh..."
2,Good as Hell [Two Stacks Remix],"[I do my hair toss, Check my nails, Baby how y...","[[hair, toss], [check, nail], [baby, feelin], ..."
3,Truth Hurts,"[Why men great 'til they gotta be great?, Woo,...","[[men, great, til, gotta, great], [woo], [took..."
4,Lose You to Love Me,"[You promised the world and I fell for it, I p...","[[promised, world, fell], [put, first, adored]..."


In [84]:
max_len = 0
for index, row in df.iterrows():
    lyrics = row['preprocessed_lyrics']
    if len(lyrics) > max_len:
        max_len = len(lyrics)
        
print(max_len)

171


In [56]:
embedding_size = 100
window_size = 40
min_word = 5
down_sampling = 1e-2

'''ft_model = FastText(size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      sg=0,
                      iter=100)'''
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True)  

In [87]:
lyrics_embeddings = np.zeros((1, 300))

def line2vec(line):
    line_embedding = np.zeros(300)
    exceptions = 0
    for word in line:
        #print(model[word].shape)
        try:
            line_embedding += model[word]
        except KeyError:
            #print(word)
            exceptions += 1
    
    return line_embedding / (len(line) - exceptions)

for index, row in df.iterrows():
    lyrics = row['preprocessed_lyrics']
    
    if index % 500 == 0:
        print(index)
    
    lyric_embedding = np.zeros(300)
    for line in lyrics:
        lyric_embedding += line2vec(line)
        
    lyric_embedding /= len(lyrics)
    
    lyric_embedding = np.reshape(lyric_embedding, (1, -1))
    #print(lyrics_embeddings.shape)
    #print(lyric_embedding.shape)
    lyrics_embeddings = np.concatenate((lyrics_embeddings, lyric_embedding), axis=0)
    
print(len(lyrics))
lyrics_embeddings = np.delete(lyrics_embeddings, [0], axis=0)

0


C:\Users\jolee\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
45


In [88]:
np.save('embeddings.npy', lyrics_embeddings)
lyrics_embeddings

array([[ 0.07220647,  0.04275346, -0.00998778, ..., -0.05670846,
         0.04547378,  0.00211467],
       [ 0.03941904,  0.04168566,  0.03906348, ..., -0.08809669,
        -0.01064252, -0.01939874],
       [ 0.0327902 ,  0.05349441, -0.01711753, ..., -0.02468307,
        -0.00638571, -0.00779375],
       ...,
       [ 0.07453166,  0.02165329, -0.01343231, ..., -0.0873936 ,
        -0.01792668, -0.02258577],
       [ 0.03994652,  0.03598347,  0.008056  , ..., -0.06673159,
         0.03109074, -0.06365595],
       [-0.01116084, -0.03000093,  0.04989477, ..., -0.0744406 ,
        -0.01147426,  0.01556036]])

In [89]:
test = np.load('embeddings.npy')
test

array([[ 0.07220647,  0.04275346, -0.00998778, ..., -0.05670846,
         0.04547378,  0.00211467],
       [ 0.03941904,  0.04168566,  0.03906348, ..., -0.08809669,
        -0.01064252, -0.01939874],
       [ 0.0327902 ,  0.05349441, -0.01711753, ..., -0.02468307,
        -0.00638571, -0.00779375],
       ...,
       [ 0.07453166,  0.02165329, -0.01343231, ..., -0.0873936 ,
        -0.01792668, -0.02258577],
       [ 0.03994652,  0.03598347,  0.008056  , ..., -0.06673159,
         0.03109074, -0.06365595],
       [-0.01116084, -0.03000093,  0.04989477, ..., -0.0744406 ,
        -0.01147426,  0.01556036]])